# Import Library


---


---
Library yang dipakai akan di-input di awal sebelum program dijalankan.



In [1]:
# @title
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import numpy as np
from IPython.display import clear_output
import uuid
from datetime import datetime
import pytz
import json
from collections import defaultdict, Counter, deque

# Read File Dataset & Kamus Case


---
---

Dataset yang digunakan adalah banyak model dari pabrikan motor yang dijual di Indonesia, dan untuk kasus lampau yang pernah disimpan akan ditampung pada sebuah file JSON beserta metadata. Pada kasus ini, disimpan di dalam Google Drive untuk mempermudah akses file.

---

untuk membuat json, bisa sediakan file kosong (misal: case-base.json) dengan isi list kosong ( [ ] )

In [3]:
# @title
df=pd.read_excel("/content/data_motor_excel_update1.xlsx")
json_path = '/content/drive/MyDrive/Bekas Kuliah/Tugas Akhir/File berkas pendukung/case_base.json'

# Proses Pengecekan Dataset
---
---
Sebelum dataset digunakan, pengecekan dilakukan untuk melihat apakah terdapat data yang kosong/tidak valid untuk menghindari hasil rekomendasi yang kurang bagus.



1.  **df.head(5)**



---

*   checking isi 5 data teratas pada dataset





In [8]:
# @title
df.head(5)

,Brand,Category,Model,Transmission,ClutchType,Bore,Stroke,PistonCount,Displacement,PowerHP,EngineConfig,FuelTank,WeightKG,FuelConsumptionKML,Price
0,Yamaha,MaticSport,aerox 155,Automatic,Dry,58.0,58.7,1,155.09,15.15,NearSquare,5.5,122.0,40.0,28330000
1,Yamaha,MaticSport,nmax 155,Automatic,Dry,58.0,58.7,1,155.09,15.15,NearSquare,7.1,130.0,40.0,32175000
2,Yamaha,MaticSport,lexi 155,Automatic,Dry,58.0,58.7,1,155.09,15.15,NearSquare,4.2,116.0,41.0,26280000
3,Yamaha,MaticSport,xmax 250,Automatic,Dry,70.0,64.9,1,249.77,22.53,OverBore,13.0,181.0,28.0,67965000
4,Yamaha,MaticClassic,grand filano,Automatic,Dry,52.4,57.9,1,124.86,8.18,OverStroke,4.4,100.0,60.0,27965000




2.  **df.info()**
---


*   lihat info data excel




In [9]:
# @title
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Brand               181 non-null    string 
 1   Category            181 non-null    string 
 2   Model               181 non-null    string 
 3   Transmission        181 non-null    string 
 4   ClutchType          181 non-null    string 
 5   Bore                181 non-null    float64
 6   Stroke              181 non-null    float64
 7   PistonCount         181 non-null    int64  
 8   Displacement        181 non-null    float64
 9   PowerHP             181 non-null    float64
 10  EngineConfig        181 non-null    string 
 11  FuelTank            181 non-null    float64
 12  WeightKG            181 non-null    float64
 13  FuelConsumptionKML  181 non-null    float64
 14  Price               181 non-null    int64  
dtypes: float64(7), int64(2), string(6)
memory usage: 21.3 KB




3. **df.isnull().sum()**


---


*   Untuk mengecek apakah terdapat data yang kosong pada dataset





In [10]:
# @title
df.isnull().sum()

,0
Brand,0
Category,0
Model,0
Transmission,0
ClutchType,0
Bore,0
Stroke,0
PistonCount,0
Displacement,0
PowerHP,0


4. **Pengubahan tipe data dari 'object' menjadi 'string'**


---



In [11]:
# @title
df['Brand'] = df['Brand'].astype('string')
df['Category'] = df['Category'].astype('string')
df['Model'] = df['Model'].astype('string')
df['Transmission'] = df['Transmission'].astype('string')
df['ClutchType'] = df['ClutchType'].astype('string')
df['EngineConfig'] = df['EngineConfig'].astype('string')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Brand               181 non-null    string 
 1   Category            181 non-null    string 
 2   Model               181 non-null    string 
 3   Transmission        181 non-null    string 
 4   ClutchType          181 non-null    string 
 5   Bore                181 non-null    float64
 6   Stroke              181 non-null    float64
 7   PistonCount         181 non-null    int64  
 8   Displacement        181 non-null    float64
 9   PowerHP             181 non-null    float64
 10  EngineConfig        181 non-null    string 
 11  FuelTank            181 non-null    float64
 12  WeightKG            181 non-null    float64
 13  FuelConsumptionKML  181 non-null    float64
 14  Price               181 non-null    int64  
dtypes: float64(7), int64(2), string(6)
memory usage: 21.3 KB


# Preprocessing


---



---

Pada proses ini, dataset mentah akan dilakukan one-hot encoding pada data kategorikal (berupa string) dan juga generalisasi data numerik dengan metode min-max.



*   One-hot encoding dilakukan agar data kategorikal bisa diubah menjadi data numerikal yang akan digunakan pada proses perhitungan cosine similarity. Data kategorikal yang berubah menjadi numerikal akan bernilai 1 dan 0. Angka 1 menyatakan "iya" dan 0 menyatakan "tidak".
*   Generalisasi angka dilakukan agar data numerikal memiliki nilai yang lebih mudah diolah pada proses perhitungan cosine similarity. Cosine similarity cukup sensitif dengan angka yang tidak dibuat adil, misal seperti harga yang bisa memiliki banyak angka nol untuk menyatakan nilai harga (misal antara juta dan miliar). Dengan menggunakan metode min-max, rentang atribut harga dari paling kecil hingga paling besar akan berurutan dengan value diantara 0 dan 1. Hasil dari generalisasi yang mendekati angka 0 berarti memiliki value yang lebih rendah, pula sebaliknya.



In [12]:
# @title
# Pilih kolom kategori untuk One-Hot Encoding
categorical_cols = ["Brand", "Category", "Transmission", "ClutchType", "EngineConfig"]

# One-Hot Encoding
encoder = OneHotEncoder(sparse_output=False, drop=None)
encoded_data = encoder.fit_transform(df[categorical_cols])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))

# Pilih kolom numerikal untuk normalisasi
numerical_cols = ["Price", "Displacement", "FuelConsumptionKML", "FuelTank", "WeightKG", "PowerHP"]

# Normalisasi dengan Min-Max Scaling
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df[numerical_cols]), columns=[col + "_normalized" for col in numerical_cols])

# Gabungkan hasil One-Hot Encoding dan Normalisasi ke dalam dataset tanpa menghapus data mentah
final_df = pd.concat([df, df_scaled, encoded_df], axis=1)

# Preview hasil
final_df.head(5)

,Brand,Category,Model,Transmission,ClutchType,Bore,Stroke,PistonCount,Displacement,PowerHP,...,Category_SuperSportNaked,Category_Touring,Transmission_Automatic,Transmission_DCT,Transmission_Manual,ClutchType_Dry,ClutchType_Wet,EngineConfig_NearSquare,EngineConfig_OverBore,EngineConfig_OverStroke
0,Yamaha,MaticSport,aerox 155,Automatic,Dry,58.0,58.7,1,155.09,15.15,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,Yamaha,MaticSport,nmax 155,Automatic,Dry,58.0,58.7,1,155.09,15.15,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,Yamaha,MaticSport,lexi 155,Automatic,Dry,58.0,58.7,1,155.09,15.15,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,Yamaha,MaticSport,xmax 250,Automatic,Dry,70.0,64.9,1,249.77,22.53,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,Yamaha,MaticClassic,grand filano,Automatic,Dry,52.4,57.9,1,124.86,8.18,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


# Kumpulan Fungsi pada Sistem
---
---
Fungsi-fungsi yang digunakan pada sistem di-define terlebih dahulu disini agar perubahan terhadap satu fungsi tidak akan mengakibatkan kerusakan secara menyeluruh (modular).

> Case-vector
* Pada fungsi ini, dijalankan sebuah perintah yang membuat keseluruhan dataset menjadi sebuah vector. Namun, terdapat sebuah jalur pintas yakni mengubah dataset menjadi satu matrix besar daripada menghitung banyak vector menggunakan loop saat proses menghitung cosine similarity. Matrix case ini bisa digunakan oleh library cosine similarity untuk mempercepat proses pencarian nilai kemiripan, dibanding membandingkan keinginan user dengan satu vector data motor secara looping.
* using: raw dataset

In [13]:
# @title
drop_columns = [
    "Brand", "Category", "Model", "Transmission", "ClutchType",
    "Bore", "Stroke", "PistonCount",
    "Displacement", "PowerHP", "EngineConfig",
    "FuelTank", "WeightKG", "FuelConsumptionKML", "Price"
]

case_vector_df = final_df.drop(columns=drop_columns)
case_matrix = case_vector_df.to_numpy()

print("Case matrix shape:", case_matrix.shape)
print("Contoh case vector pertama:", case_matrix[0])

Case matrix shape: (181, 45)
Contoh case vector pertama: [0.00959656 0.02531241 0.54098361 0.0719697  0.10365854 0.03349282
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 1.         0.         0.         0.         0.         0.
 0.         1.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         1.         0.         0.         1.         0.
 1.         0.         0.        ]


> User-vector & Weight-vector
* Input/keinginan user akan dikonversi menjadi sebuah vector terlebih dahulu agar bisa diolah pada algoritma cosine similarity. Weigth-vector berfungsi sebagai pemandu arah agar keinginan user memiliki bobot prioritas yang sudah ditentukan di awal pada proses input oleh user.
* User-vector akan menjadi acuan data/vector yang akan dicari di case-vector (dalam bentuk matrix/kumpulan vector), sedangkan weight-vector adalah bobot yang akan menentukan prioritas user-vector.
* using: user's input

In [14]:
# @title
def buat_user_vector_weighted(user_input, final_df, df_mentah):
    """
    Buat user_vector dan weight_vector berdasarkan preferensi user,
    mempertimbangkan urutan prioritas (semakin awal = bobot lebih besar).
    """
    user_vector = np.zeros(len(final_df.columns), dtype=float)
    weight_vector = np.zeros(len(final_df.columns), dtype=float)
    col_index = {col: i for i, col in enumerate(final_df.columns)}

    for i, (attr, val) in enumerate(user_input):
        weight = 1 / (i + 1)  # <<<<<<<<-----------Bobot menurun sesuai urutan prioritas

        # One-hot encoding
        if attr in ["Brand", "Category", "Transmission", "ClutchType", "EngineConfig"]:
            prefix = attr.replace(" ", "")  # Kolom hasil one-hot biasanya tanpa spasi
            col_name = f"{prefix}_{val}".lower()

            # Cari kolom match di final_df (case insensitive)
            match = [col for col in final_df.columns if col.lower() == col_name]
            if match:
                idx = col_index[match[0]]
                user_vector[idx] = 1.0
                weight_vector[idx] = weight

        # Numerikal
        else:
            norm_col = f"{attr}_normalized"
            if norm_col in final_df.columns and attr in df_mentah.columns:
                idx = col_index[norm_col]
                max_val = df_mentah[attr].max()
                norm_val = float(val) / max_val
                user_vector[idx] = norm_val
                weight_vector[idx] = weight

    return user_vector, weight_vector

> Case-based Recommendation System
* Fungsi ini akan mencocokan vector user yang sudah ditampung dengan case vector dalam bentuk matriks. Hasil akhir dari fungsi ini adalah rekomendasi model motor yang mendekati keinginan user.
* using: case-vector, user-vector & weight-vector

In [15]:
# @title
def rekomendasi_cosine_weighted(user_vec, weight_vec, case_matrix, final_df, user_input, top_n=10):
    """
    Menghitung cosine similarity berbobot + penalti selisih nilai numerik dari preferensi user.
    """
    # Konversi preferensi user ke dict
    user_pref = dict(user_input)

    # Siapkan target numerik dari preferensi user (jika ada)
    target_power = float(user_pref.get("PowerHP", 0))
    target_cc = float(user_pref.get("Displacement", 0))
    target_price = float(user_pref.get("Price", 0))
    target_weight = float(user_pref.get("WeightKG", 0))
    target_fuel = float(user_pref.get("FuelTank", 0))

    # Hitung cosine similarity (berbobot)
    weighted_user_vector = user_vec * weight_vec
    weighted_case_matrix = case_matrix * weight_vec
    similarity_scores = cosine_similarity([weighted_user_vector], weighted_case_matrix)[0]

    # Tempel ke final_df
    final_df_with_score = final_df.copy()
    final_df_with_score["Similarity"] = similarity_scores

    # Hitung penalti
    final_df_with_score["PowerPenalty"] = abs(final_df_with_score["PowerHP"] - target_power) if target_power else 0
    final_df_with_score["CCPenalty"] = abs(final_df_with_score["Displacement"] - target_cc) if target_cc else 0
    final_df_with_score["PricePenalty"] = abs(final_df_with_score["Price"] - target_price) / 1_000_000 if target_price else 0
    final_df_with_score["WeightPenalty"] = abs(final_df_with_score["WeightKG"] - target_weight) if target_weight else 0
    final_df_with_score["FuelPenalty"] = abs(final_df_with_score["FuelTank"] - target_fuel) if target_fuel else 0

    # Final score: cosine - penalti (atur skala sesuai preferensi) <<<<<------ buat atur skala prioritas numerikal
    final_df_with_score["FinalScore"] = (
        final_df_with_score["Similarity"]
        - 0.04 * final_df_with_score["PowerPenalty"]
        - 0.02 * final_df_with_score["CCPenalty"]
        - 0.01 * final_df_with_score["PricePenalty"]
        - 0.01 * final_df_with_score["WeightPenalty"]
        - 0.01 * final_df_with_score["FuelPenalty"]
    )

    # Urutkan dan ambil top-N
    sorted_df = final_df_with_score.sort_values(by="FinalScore", ascending=False)
    return sorted_df.head(top_n)

> Fungsi Tampilan Menu
* Fungsi ini akan menampilkan menu untuk menerima input user. Menu dibuat lebih rapih agar memudahkan user.
* using: (none)

In [16]:
# @title

# Daftar pilihan per atribut kategorikal (nomor urut otomatis)
daftar_pilihan = {
    "Brand": ["Yamaha", "Honda", "Kawasaki", "Suzuki", "Benelli", "RoyalEnfield", "HarleyDavidson",
              "CFMOTO", "Keeway", "BMW", "TVS", "Ducati"],
    "Category": ["MaticSport", "MaticClassic", "MaticDaily", "Moped", "MiniBike", "SportRetro",
                 "SportNaked", "SportFairing", "SportAdventure", "SuperSportFairing",
                 "SuperSportNaked", "HyperSportFairing", "DualSport/Trail", "Cruiser",
                 "RetroClassic", "Touring"],
    "Transmission": ["Automatic", "Manual"],
    "ClutchType": ["Wet", "Dry", "DCT"],
    "EngineConfig": ["OverBore", "OverStroke", "NearSquare"]
}

# Mapping nomor ke nama atribut dan pertanyaan
atribut_info = {
    "1": ("Brand", "Pilih nomor brand motor yang diinginkan: "),
    "2": ("Category", "Pilih nomor kategori motor yang diinginkan: "),
    "3": ("Transmission", "Pilih nomor jenis transmisi motor yang diinginkan: "),
    "4": ("ClutchType", "Pilih nomor jenis kopling motor yang diinginkan: "),
    "5": ("Displacement", "Target kapasitas mesin (cc), (110 - 2000): "),
    "6": ("PowerHP", "Target tenaga motor (Horsepower), (10 - 240): "),
    "7": ("EngineConfig", "Pilih nomor konfigurasi mesin yang diinginkan: "),
    "8": ("FuelTank", "Target kapasitas tangki (dalam liter, 4 - 40): "),
    "9": ("WeightKG", "Berat motor target (dalam kg, 60 - 200): "),
    "10": ("FuelConsumptionKML", "Target konsumsi BBM (km/l): "),
    "11": ("Price", "Harga motor target (dalam rupiah): ")
}

user_priority_queue = deque()

# Fungsi: tampilkan menu utama
def tampilkan_menu():
    print("\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
    print("Input Urutan Kebutuhan Motor")
    print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")

    # Tampilkan isi antrian sebelum menu
    if user_priority_queue:
        print("\n📋 Kebutuhan yang sudah diinput:")
        for i, (a, v) in enumerate(user_priority_queue, 1):
            print(f"{i}. {a} = {v}")
        print("\n" + "\\" * 50 + "\n")

    for k, v in atribut_info.items():
        print(f"{k}. {v[0]}")
    print("0. Hapus input terakhir")
    print("99. Selesai input dan lanjutkan")

# Fungsi: tampilkan daftar pilihan untuk atribut kategorikal dan ambil berdasarkan nomor input
def pilih_dari_opsi(nama_atribut):
    opsi = daftar_pilihan[nama_atribut]
    print("\n" + "*" * 45)
    print(f"Pilihan untuk {nama_atribut}:")
    for i, opsi_item in enumerate(opsi, 1):
        print(f"{i}. {opsi_item}")
    print("*" * 45)

    while True:
        try:
            pilihan_nomor = int(input(f"Masukkan nomor pilihan (1-{len(opsi)}): "))
            if 1 <= pilihan_nomor <= len(opsi):
                return opsi[pilihan_nomor - 1]
            else:
                print("❌ Nomor di luar rentang. Coba lagi.")
        except ValueError:
            print("❌ Input harus berupa angka. Coba lagi.")

> Format Output/Tampilan hasil rekomendasi awal
* Fungsi ini berfungsi menampilkan hasil rekomendasi agar lebih mudah dilihat oleh user.
* using: modified dataset

In [17]:
# @title
def print_hasil(df_hasil):
    print("📌 Rekomendasi Motor Berdasarkan Preferensi Kamu:\n")

    for nomor, (idx, row) in enumerate(df_hasil.iterrows(), 1):  # <- mulai dari 1
        is_top1 = (nomor == 1)
        is_top2 = (nomor == 2)

        if is_top1:
            print("⭐ TOP-1 REKOMENDASI ⭐")
            print("👉 Motor ini adalah yang paling mendekati preferensimu berdasarkan perhitungan sistem.\n")

        if is_top2:
            print("\n")
            print("⭐ Kandidat rekomendasi motor lainnya ⭐")
            print("👉 Hasil rekomendasi 5 motor di bawah ini memiliki tingkat kemiripan yang masih dekat dengan preferensi awalmu.\n")

        print(f"📌 Nomor urut motor: ke-{nomor}")
        print(f"🏍️  {row['Model']} | {row['Brand']}")
        print(f"    - Category: {row['Category']}")
        print(f"    - Transmission: {row['Transmission']}")
        print(f"    - Power: {row['PowerHP']} HP")
        print(f"    - Displacement: {row['Displacement']} cc")
        print(f"    - Engine Config: {row['EngineConfig']}")
        print(f"    - Fuel Tank: {row['FuelTank']} L")
        print(f"    - Weight: {row['WeightKG']} kg")
        print(f"    - Price: Rp{int(row['Price']):,}")
        print(f"    - ⚙️ Similarity Score: {row['Similarity']:.4f}")
        print(f"    - 🎯 Final Score (w/ penalty): {row['FinalScore']:.4f}")
        print("-" * 60)

> Fungsi Auto Generated case_id + timestamp lokal Indonesia
* Fungsi ini memberikan case_id yang unik pada metadata dengan menggunakan waktu lokal indonesia + kode unik. Sedangkan fungsi timestamp untuk menyimpan data waktu kapan case disimpan.
* using: (none)

In [18]:
# @title
def timestamp_WIB():
    return datetime.now(pytz.timezone('Asia/Jakarta')).strftime("%Y-%m-%d %H:%M:%S")

def generate_case_id():
    return f"case_{datetime.now(pytz.timezone('Asia/Jakarta')).strftime('%Y%m%d%H%M%S')}_{uuid.uuid4().hex[:6]}"

> Fungsi kirim case ke dalam kamus case lampau (file JSON)
* Fungsi ini hanya berfungsi mengirim data ke file tujuan, yaitu file json untuk menampung case lampau.
* using: (none)

In [19]:
# @title
def kirim_case_ke_library(case_data, path=json_path):
    # Load file lama
    try:
        with open(path, 'r') as f:
            case_base = json.load(f)
    except FileNotFoundError:
        case_base = []

    # Tambah case baru
    case_base.append(case_data)

    # Simpan ulang
    with open(path, 'w') as f:
        json.dump(case_base, f, indent=4)

    print("✅ Case baru berhasil disimpan di library.")

> Fungsi tampung metadata kasus dan kirim kasus
* Fungsi ini menampung metadata hasil kasus yang ingin disimpan, dan metadata tersebut dikirim menggunakan fungsi simpan_case()
* using: modified dataset, simpan_case(), auto generate case_id + timestamp

In [20]:
# @title
def simpan_case_model(user_input, row_model, refined=False, refine_log=None, user_ranked=False):
    case_data = {
        "case_id": generate_case_id(),
        "user_input": user_input,
        "is_refined": refined,
        "refine_steps": refine_log if refine_log else [],
        "refine_iteration_count": len(refine_log) if refine_log else 0,
        "chosen_models": [{
            "model": row_model["Model"],
            "similarity_score": float(row_model["Similarity"])
        }],
        "user_ranked": user_ranked,
        "timestamp": datetime.now(pytz.timezone("Asia/Jakarta")).strftime("%Y-%m-%d %H:%M:%S")
    }
    kirim_case_ke_library(case_data)

> Fungsi cari kriteria case pada kamus case lampau
* Fungsi ini mencari kriteria atribut yang telah disebutkan oleh user di kamus case lampau.
* using: json_path, user's input

In [21]:
# @title
def cari_case_lama(case_base_path, user_input):
    try:
        with open(case_base_path, 'r') as f:
            case_base = json.load(f)
    except FileNotFoundError:
        return []

    hasil = []
    user_input_set = set((k.lower(), str(v).strip().lower()) for k, v in user_input)

    for case in case_base:
        case_input = set((k.lower(), str(v).strip().lower()) for k, v in case.get("user_input", []))
        if case_input == user_input_set:
            hasil.append(case)

    return hasil

> Fungsi load cases pada kamus case lampau
* Fungsi ini menampilkan casus lama yang sama dengan kriteria yang disebutkan oleh user di awal.
* using: json_path, user's input

In [22]:
# @title
def tampilkan_case_tergabung(case_base_path, user_input):
    try:
        with open(case_base_path, 'r') as f:
            case_base = json.load(f)
    except FileNotFoundError:
        print("❌ Tidak ada case base ditemukan.")
        return

    def normalisasi_input(inp):
        return set((k.lower(), str(v).strip().lower()) for k, v in inp)

    target_input = normalisasi_input(user_input)

    matched_cases = [
        case for case in case_base
        if normalisasi_input(case.get("user_input", [])) == target_input
    ]

    if not matched_cases:
        print("❌ Tidak ditemukan case serupa sebelumnya di memori sistem.\n")
        return

    # Hitung model dan breakdown pemilih
    model_counter = Counter()
    user_count = defaultdict(int)
    system_count = defaultdict(int)

    for case in matched_cases:
        for model in case.get("chosen_models", []):
            nama = model["model"]
            model_counter[nama] += 1
            if case.get("user_ranked", False):
                user_count[nama] += 1
            else:
                system_count[nama] += 1

    print(f"📚 Ditemukan {len(matched_cases)} kasus sebelumnya dengan preferensi yang sama:\n")
    print("🔁 Preferensi:")
    for attr, val in user_input:
        print(f"   - {attr} = {val}")

    print("\n📌 Model yang pernah dipilih sebelumnya:")
    for model, count in model_counter.items():
        sys_c = system_count.get(model, 0)
        user_c = user_count.get(model, 0)
        print(f"   - {model} (dipilih {count}x, {sys_c}x oleh sistem, {user_c}x oleh user)")

    print("-" * 60 + "\n")


proses refinement

In [23]:
# @title
def refine_preferences_queue(user_input, top1_motor):
    print("\n🔁 Proses refinement ini merujuk pada motor Top-1 terbaru:")
    print(f"📌 Model referensi: {top1_motor['Model']}")

    refine_queue = []

    atribut_numerik = {
        "Displacement": "Kapasitas mesin (cc)",
        "PowerHP": "Tenaga (Horsepower)",
        "FuelTank": "Kapasitas tangki (liter)",
        "WeightKG": "Berat motor (kg)",
        "FuelConsumptionKML": "Konsumsi BBM (km/l)",
        "Price": "Harga motor (rupiah)"
    }

    atribut_kategorikal = {
        "Brand": ["Yamaha", "Honda", "Kawasaki", "Suzuki", "Benelli", "RoyalEnfield", "HarleyDavidson",
                  "CFMOTO", "Keeway", "BMW", "TVS", "Ducati"],
        "Category": ["MaticSport", "MaticClassic", "MaticDaily", "Moped", "MiniBike", "SportRetro",
                     "SportNaked", "SportFairing", "SportAdventure", "SuperSportFairing",
                     "SuperSportNaked", "HyperSportFairing", "DualSport/Trail", "Cruiser",
                     "RetroClassic", "Touring"],
        "Transmission": ["Automatic", "Manual"],
        "ClutchType": ["Wet", "Dry", "DCT"],
        "EngineConfig": ["OverBore", "OverStroke", "NearSquare"]
    }

    all_atribut = list(atribut_numerik.keys()) + list(atribut_kategorikal.keys())
    initial_input_attributes = [a for a, _ in user_input]

    while True:
        print("\n📋 Atribut yang bisa di-refine:")
        for idx, attr in enumerate(all_atribut, 1):
            if attr in initial_input_attributes:
                current_val = next((val for a, val in user_input if a == attr), top1_motor.get(attr, "-"))
            else:
                current_val = "as it is / tidak berubah"

            future_val = next((val for a, val in refine_queue if a == attr), None)

            if future_val is not None:
                icon = "✏️"
                print(f"{idx}. {attr} {icon} (current: {current_val} → update to: {future_val})")
            else:
                print(f"{idx}. {attr} (current: {current_val})")

        print("0. Selesai memilih perubahan")
        print("99. Hapus perubahan yang sudah ditampung")

        try:
            pilihan = int(input("\nMasukkan nomor pilihan: "))
            if pilihan == 0:
                break

            if pilihan == 99:
                if not refine_queue:
                    print("⚠️ Belum ada perubahan untuk dihapus.")
                    continue
                print("\n📦 Daftar perubahan yang bisa dihapus:")
                for i, (a, v) in enumerate(refine_queue, 1):
                    print(f"{i}. {a} → {v}")
                hapus_idx = input("Masukkan nomor perubahan yang ingin dihapus (atau tekan Enter untuk batal): ")
                if hapus_idx.strip() == "":
                    continue
                try:
                    idx_to_remove = int(hapus_idx)
                    if 1 <= idx_to_remove <= len(refine_queue):
                        removed = refine_queue.pop(idx_to_remove - 1)
                        print(f"🗑️ Dihapus: {removed[0]} → {removed[1]}")
                    else:
                        print("❌ Nomor tidak valid.")
                except ValueError:
                    print("❌ Harus angka.")
                continue

            if 1 <= pilihan <= len(all_atribut):
                attr = all_atribut[pilihan - 1]

                if attr in atribut_numerik:
                    current_val = next((val for a, val in user_input if a == attr), top1_motor[attr])

                    print(f"\nApa yang ingin kamu lakukan untuk '{attr}'?")
                    print("1. Tambahkan 10%")
                    print("2. Kurangi 10%")
                    print("3. Masukkan nilai manual")

                    opsi = input("Masukkan pilihanmu (1/2/3): ")

                    if opsi == "1":
                        new_val = round(current_val * 1.10, 2)
                    elif opsi == "2":
                        new_val = round(current_val * 0.90, 2)
                    elif opsi == "3":
                        new_val = float(input(f"Masukkan nilai baru untuk {attr}: "))
                    else:
                        print("❌ Pilihan tidak valid.")
                        continue

                    refine_queue = [item for item in refine_queue if item[0] != attr]
                    refine_queue.append((attr, new_val))
                    print(f"📝 Ditampung: {attr} → {new_val}")

                elif attr in atribut_kategorikal:
                    current_val = next((val for a, val in user_input if a == attr), top1_motor.get(attr, "-"))
                    opsi = atribut_kategorikal[attr]

                    print(f"\nPilihan {attr} (current: {current_val}):")
                    for i, item in enumerate(opsi, 1):
                        print(f"{i}. {item}")
                    pilih = int(input(f"Masukkan nomor pilihan baru untuk {attr}: "))
                    if 1 <= pilih <= len(opsi):
                        new_val = opsi[pilih - 1]
                        refine_queue = [item for item in refine_queue if item[0] != attr]
                        refine_queue.append((attr, new_val))
                        print(f"📝 Ditampung: {attr} → {new_val}")
                    else:
                        print("❌ Pilihan tidak valid.")
            else:
                print("❌ Pilihan tidak valid.")

        except ValueError:
            print("❌ Input harus berupa angka.")

    return refine_queue


fungsi write iterasi log per iterasi

In [24]:
# @title
def write_log_refine_iteration(refine_queue, user_input_before):
    step = []
    for attr, new_val in refine_queue:
        old_val = next((val for a, val in user_input_before if a == attr), None)
        step.append({
            "attribute": attr,
            "from": old_val,
            "to": new_val
        })
    return step

> Fungsi utama rekomendasi
* Program utama sistem rekomendasi menggunakan conversational case-based
* using: all.

In [25]:
# @title
def proses_evaluasi_rekomendasi(df_hasil, user_input):

    # Tampilkan hasil dari case sebelumnya
    case_lama = cari_case_lama(json_path, urutan_kebutuhan_user)

    if case_lama:
        tampilkan_case_tergabung(json_path, urutan_kebutuhan_user)
    else:
        print("❌ Tidak ditemukan case serupa sebelumnya di memori sistem.\n")

    # Tampilkan hasil rekomendasi aktual
    hasil_6_besar = df_hasil.head(6)
    print_hasil(hasil_6_besar)

    puas = input("\nApakah hasil TOP-1 sudah sesuai dengan preferensimu? (Y/N): ").lower()

    if puas == "y":
        idx = 1
        simpan_case_model(user_input, hasil_6_besar.iloc[idx - 1], refined=False, user_ranked=False)
        print("Kepuasan hasil rekomendasimu akan kami simpan untuk future similiar case, terima kasih!")
        return "selesai"

    prefer_lain = input("\nApakah ada motor lain di urutan 2-6 yang menurutmu lebih cocok? (ketik nomor atau 'N'): ").lower()

    if prefer_lain.isdigit():
        idx = int(prefer_lain)
        if 2 <= idx <= 6:
            simpan_case_model(user_input, hasil_6_besar.iloc[idx - 1], refined=False, user_ranked=False)
            print("Hasil pilihan rekomendasimu akan kami simpan untuk future similiar case, terima kasih!")
            return "selesai"

    # Mulai refinement
    if prefer_lain == "n":
        lanjut_refine = input("Apakah kamu ingin memodifikasi kriteria agar hasil lebih sesuai? (Y/N): ").lower()
        if lanjut_refine != "y":
            print("❌ Sistem selesai. Case tidak disimpan karena tidak ditemukan hasil yang memuaskan.")
            return "selesai"

        user_refined = user_input.copy()
        refine_queue = []
        refine_history = []

        while True:
            refine_queue = refine_preferences_queue(user_refined, hasil_6_besar.iloc[0])
            if not refine_queue:
                print("⚠️ Tidak ada perubahan yang dimasukkan. Refinement dibatalkan.")
                return "selesai"

            print("\n📋 Perubahan yang akan diterapkan:")
            for attr, val in refine_queue:
                print(f" - {attr} → {val}")

            konfirmasi = input("Terapkan perubahan ini dan hitung ulang rekomendasi? (Y/N): ").lower()
            if konfirmasi != "y":
                print("❌ Perubahan dibatalkan.")
                continue

            # # Terapkan perubahan ke user_refined
            # for attr, val in refine_queue:
            #     updated = False
            #     for i, (a, _) in enumerate(user_refined):
            #         if a == attr:
            #             user_refined[i] = (a, val)
            #             updated = True
            #             break
            #     if not updated:
            #         user_refined.append((attr, val))

            # ✅ 1. Catat iterasi refinement
            refine_step = write_log_refine_iteration(refine_queue, user_refined)
            refine_history.append(refine_step)

            # ✅ 2. Terapkan perubahan ke user_refined
            for attr, val in refine_queue:
                updated = False
                for i, (a, _) in enumerate(user_refined):
                    if a == attr:
                        user_refined[i] = (a, val)
                        updated = True
                        break
                if not updated:
                    user_refined.append((attr, val))

            refine_queue.clear()


            # Hitung ulang rekomendasi
            user_vec, weight_vec = buat_user_vector_weighted(user_refined, case_vector_df, df)
            hasil_refined = rekomendasi_cosine_weighted(user_vec, weight_vec, case_matrix, final_df, user_refined, top_n=10)
            hasil_6_baru = hasil_refined.head(6)

            print("\n🔁 Rekomendasi setelah refinement:")
            print_hasil(hasil_6_baru)

            puas_refine = input("\nApakah motor urutan pertama dari hasil refinement sudah sesuai? (Y/N): ").lower()
            if puas_refine == "y":
                simpan_case_model(
                    user_refined,
                    hasil_6_baru.iloc[0],
                    refined=True,
                    refine_log=refine_history,
                    user_ranked=False,
                )
                print("✅ Hasil refined rekomendasi telah disimpan.")
                return "selesai"

            prefer_lain_refine = input("Apakah ada motor lain di urutan 2–6 hasil refinement yang lebih cocok? (ketik nomor atau 'N'): ").lower()
            if prefer_lain_refine.isdigit():
                idx = int(prefer_lain_refine)
                if 2 <= idx <= 6:
                    simpan_case_model(
                        user_refined,
                        hasil_6_baru.iloc[idx - 1],
                        refined=True,
                        refine_log=refine_history,
                        user_ranked=True,
                    )
                    print("✅ Pilihan refined telah disimpan.")
                    return "selesai"

            lanjut_refine = input("Ingin melakukan refinement lagi? (Y/N): ").lower()
            if lanjut_refine != "y":
                print("❌ Refinement dihentikan. Case tidak disimpan.")
                return "selesai"


# Aplikasi Sistem Rekomendasi Motor menggunakan Conversational Case-based
---
---

> Proses 1: Tampung kriteria user
* Kriteria user akan di-define disini dan memiliki urutan prioritas

In [26]:
# @title
# Loop input
while True:
    tampilkan_menu()
    tampung = input("\n \nMasukkan nomor atribut yang diinginkan: ")

    if tampung == "0":
        if user_priority_queue:
            removed = user_priority_queue.pop()
            print(f"🗑️  Input terakhir '{removed[0]} = {removed[1]}' dihapus dari antrian.")
        else:
            print("⚠️  Antrian kosong, tidak ada yang bisa dihapus.")

    elif tampung == "99":
        print("\n✅ Input selesai. Urutan kebutuhan user:")
        for i, (attr, value) in enumerate(user_priority_queue, 1):
            print(f"{i}. {attr} = {value}")
        break

    elif tampung in atribut_info:
        attr_name, question = atribut_info[tampung]

        # Cek apakah atribut ini sudah pernah diinput
        if attr_name in [item[0] for item in user_priority_queue]:
            print(f"\n⚠️⚠️⚠️  Atribut '{attr_name}' sudah ada di antrian. Pilih atribut lain. \n\n")
            continue

        # Proses input berdasarkan tipe atribut
        if attr_name in daftar_pilihan:
            user_value = pilih_dari_opsi(attr_name)
        else:
            user_value = input(question)
            try:
                user_value = float(user_value)
            except ValueError:
                print("❌ Input harus berupa angka. Coba lagi.")
                continue

        user_priority_queue.append((attr_name, user_value))
        print(f"✅ Atribut '{attr_name}' = '{user_value}' ditambahkan ke antrian. \n\n")

    else:
        print("❌ Input tidak valid. Coba lagi.")


# Clear menu + iterasi data yang sudah dipilih
clear_output(wait=True)
print("\n✅ Input selesai. Urutan kebutuhan user:")
for i, (attr, value) in enumerate(user_priority_queue, 1):
  print(f"{i}. {attr} = {value}")


# Akhir: variabel hasil
urutan_kebutuhan_user = list(user_priority_queue)


✅ Input selesai. Urutan kebutuhan user:
1. Brand = Yamaha
2. Category = SportNaked
3. EngineConfig = OverBore




---



> Proses 2: Generalisasi nilai dan assign bobot prioritas pada kriteria user
* Input user yang sudah ditampung, diubah menjadi nilai yang siap guna dan menyimpan bobot prioritas.

In [27]:
# assign data olahan dari fungsi vector pada sebuah variable
user_vec, weight_vec = buat_user_vector_weighted(urutan_kebutuhan_user, case_vector_df, df)

# @title
# Cetak posisi non-zero dari user_vector
print("🎯 NON-ZERO di User Vector (yang aktif karena input user/selected features):\n")
for i, val in enumerate(user_vec):
    if val != 0:
        print(f"{case_vector_df.columns[i]:<40} = {val:.3f} (weight: {weight_vec[i]:.3f})")

# Cetak total jumlah kolom aktif
print(f"\n🧠 Total fitur aktif: {np.count_nonzero(user_vec)} dari {len(user_vec)} fitur.")

🎯 NON-ZERO di User Vector (yang aktif karena input user/selected features):

Brand_Yamaha                             = 1.000 (weight: 1.000)
Category_SportNaked                      = 1.000 (weight: 0.500)
EngineConfig_OverBore                    = 1.000 (weight: 0.333)

🧠 Total fitur aktif: 3 dari 45 fitur.


---

> Proses 3: Hasil rekomendasi dan iterasi perbaikan
* Alur program --->
* User memberikan kriteria yang diinginkan (system's target)
* sistem memberikan rekomendasi paling relevan dengan kriteria yang diinginkan
* user evaluasi hasil rekomendasi.
* bila user "puas", maka sistem akan menyimpan hasil rekomendasi ke kamus kasus untuk penggunaan future case.
* bila user "tidak puas", maka sistem akan menyajikan iterasi perbaikan rekomendasi hingga memenuhi keinginan user.

In [28]:
# @title
hasil = rekomendasi_cosine_weighted(user_vec, weight_vec, case_matrix, final_df, urutan_kebutuhan_user, top_n=6)
proses_evaluasi_rekomendasi(hasil, urutan_kebutuhan_user)

❌ Tidak ditemukan case serupa sebelumnya di memori sistem.

📌 Rekomendasi Motor Berdasarkan Preferensi Kamu:

⭐ TOP-1 REKOMENDASI ⭐
👉 Motor ini adalah yang paling mendekati preferensimu berdasarkan perhitungan sistem.

📌 Nomor urut motor: ke-1
🏍️  mt-25 | Yamaha
    - Category: SportNaked
    - Transmission: Manual
    - Power: 35.4 HP
    - Displacement: 249.38 cc
    - Engine Config: OverBore
    - Fuel Tank: 14.0 L
    - Weight: 169.0 kg
    - Price: Rp63,500,000
    - ⚙️ Similarity Score: 1.0000
    - 🎯 Final Score (w/ penalty): 1.0000
------------------------------------------------------------


⭐ Kandidat rekomendasi motor lainnya ⭐
👉 Hasil rekomendasi 5 motor di bawah ini memiliki tingkat kemiripan yang masih dekat dengan preferensi awalmu.

📌 Nomor urut motor: ke-2
🏍️  mt-15 | Yamaha
    - Category: SportNaked
    - Transmission: Manual
    - Power: 19.04 HP
    - Displacement: 155.09 cc
    - Engine Config: NearSquare
    - Fuel Tank: 10.4 L
    - Weight: 133.0 kg
    - Price

'selesai'

In [ ]:
# # Simpan vektor numerik yang siap untuk cosine similarity
# case_vector_df.to_pickle("case_vector_df.pkl")

# # Simpan dataframe motor asli yang akan ditampilkan
# final_df.to_pickle("final_df.pkl")

# # Download ke lokal
# from google.colab import files
# files.download("case_vector_df.pkl")
# files.download("final_df.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>